<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [12]</a>'.</span>

In [1]:
import re, json, time
from selenium import webdriver
import os

# import By
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

# from webdriver_manager.chrome import ChromeDriverManager
from parsel import Selector

import pandas as pd
import numpy as np


# Tratamento


In [2]:
with open("canais.txt", "r") as f:
    # read each line as a item in a list and remove the \n
    canais = [line.strip() for line in f.readlines()]


def scroll_page(url):
    # service = Service(ChromeDriverManager().install())

    # options = webdriver.ChromeOptions()
    # options.add_argument("--headless")
    # options.add_argument("--lang=en")
    # options.add_argument(
    #     "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
    # )

    driver = webdriver.Chrome()
    driver.get(url)
    driver.implicitly_wait(10)

    time.sleep(3)
    selector = Selector(driver.page_source)
    driver.quit()

    return selector

In [3]:
import re


import locale

locale.setlocale(locale.LC_ALL, "pt_BR.UTF-8")

base_path = f"./data/{canais[0].split('@')[-1]}/"

print(base_path)

./data/farmacias.unipreco/


In [4]:


def trata_inscritos(col: pd.Series) -> pd.Series:
    col = col.split(" inscritos")[0]

    if "mil" in col:
        return int(
            float(col.replace("mil", "").replace(" ", "").replace(",", ".")) * 1000
        )
    if "mi" in col:
        # remove 'de'
        if "de" in col:
            col = col.replace("de", "")
        return int(
            float(col.replace("mi", "").replace(" ", "").replace(",", ".")) * 1000000
        )

    return int(col.replace(" ", "").replace(",", "."))


def trata_visualizacoes(col):
    # treat the characters U+00a0

    # if its a numeric value return it
    if type(col) == int or type(col) == float:
        return col

    col = col.replace("\xa0", "")

    col = col.split(" visualizações")[0]

    if "mil" in col:
        return int(
            float(col.replace("mil", "").replace(" ", "").replace(",", ".")) * 1000
        )
    elif "mi" in col:
        # remove 'de'
        if "de" in col:
            col = col.replace("de", "")
        return int(
            float(col.replace("mi", "").replace(" ", "").replace(",", ".")) * 1000000
        )


    try:
        return int(col.replace(" ", "").replace(".", "").replace(",", "."))
    except:
        return 0


def trata_comments(col):
    # print(col)
    # print(type(col))

    if type(col) == int or type(col) == float:
        return col
    
    value = 0

    try:
        value = int(col)
    except:
        pass


    if 'Transcrição' in col:
        return 0
    
    col = col.replace("\xa0", "")

   
    if "mil" in col:
        return int(
            float(col.replace("mil", "").replace(" ", "").replace(",", ".")) * 1000
        )
    elif "mi" in col:
        # remove 'de'
        if "de" in col:
            col = col.replace("de", "")
        try:
               value = int(
                float(col.replace("mi", "").replace(" ", "").replace(",", ".")) * 1000000
            )
        except:
            value =  0

    return int(col.replace(" ", "").replace(".", "").replace(",", ".")) if value != 0 else 0


def trata_data(value):
    meses = {
        "jan": "01",
        "fev": "02",
        "mar": "03",
        "abr": "04",
        "mai": "05",
        "jun": "06",
        "jul": "07",
        "ago": "08",
        "set": "09",
        "out": "10",
        "nov": "11",
        "dez": "12",
    }
    series_data = re.sub(r"\. de", "", value)
    # print(series_data)
    series_data = re.search(r"(\d{1,2} de [a-zA-Z]+ \d{4})", series_data).group(0)
    # replace month
    for k, v in meses.items():
        series_data = series_data.replace(k, v)

    # print(series_data)
    series_data = pd.to_datetime(series_data, format="%d de %m %Y", errors="coerce")
    return series_data




trata videos

In [5]:

videos = pd.read_csv(base_path + "videos.csv")

display(videos)
display(videos.columns)

,title,views,likes,date,comments_amount,channel_name,subscribers
0,Álbum Oficial Brasileirão 2023,149 visualizações,1,11 de set. de 2023,NaN,Farmácias Unipreço,242 inscritos
1,A MAIOR FARMÁCIA DE PALMEIRA - FARMÁCIAS UNIPREÇO,125 visualizações,1,2 de dez. de 2022,A MAIOR FARMÁCIA DE PALMEIRA - FARMÁCIAS UNIPREÇO,Farmácias Unipreço,242 inscritos
2,CHEGAMOS UNIPREÇO SÍTIO CERCADO,51 mil visualizações,0,Estreou em 20 de set. de 2022,NaN,Farmácias Unipreço,242 inscritos
3,Surpresa em comemoração aos 11 anos de Anivers...,25 visualizações,0,Estreou em 13 de set. de 2022,NaN,Farmácias Unipreço,242 inscritos
4,Comemoração 11 anos de Aniversário Farmácias U...,"1,4 mil visualizações",11,Estreou em 1 de set. de 2022,NaN,Farmácias Unipreço,242 inscritos
5,Nos Caminhos do Verão - Farmácias Nissei | EP....,174 visualizações,4,27 de fev. de 2023,Transcrição,Farmácias Nissei,"1,16 mil inscritos"
6,Nos Caminhos do Verão - Farmácias Nissei | EP....,81 visualizações,2,24 de fev. de 2023,Transcrição,Farmácias Nissei,"1,16 mil inscritos"
7,Nos Caminhos do Verão - Farmácias Nissei | EP....,49 visualizações,2,23 de fev. de 2023,Transcrição,Farmácias Nissei,"1,16 mil inscritos"
8,Nos Caminhos do Verão - Farmácias Nissei | EP....,47 visualizações,1,23 de fev. de 2023,Transcrição,Farmácias Nissei,"1,16 mil inscritos"
9,Nos Caminhos do Verão - Farmácias Nissei | EP....,32 visualizações,1,22 de fev. de 2023,Transcrição,Farmácias Nissei,"1,16 mil inscritos"


Index(['title', 'views', 'likes', 'date', 'comments_amount', 'channel_name',
       'subscribers'],
      dtype='object')

In [6]:


videos.assign(
    subscribers=lambda df_: df_["subscribers"].apply(trata_inscritos),
    views=lambda df_: df_["views"].apply(trata_visualizacoes),
    date=lambda df_: df_["date"].apply(trata_data),
    # comments_amount = lambda df_: df_["comments_amount"].apply(trata_comments),
    category="RECENT",
).to_parquet(base_path + "videos.parquet")


# Trata Top videos

In [7]:
import pandas as pd


top_videos = pd.read_csv(base_path + "top_videos.csv")

display(top_videos.head())

,title,views,likes,date,comments_amount,channel_name
0,CHEGAMOS UNIPREÇO SÍTIO CERCADO,51 mil visualizações,0,Estreou em 20 de set. de 2022,NaN,Farmácias Unipreço
1,Inauguração Mega Store Paranaguá,22 mil visualizações,5,1 de set. de 2021,NaN,Farmácias Unipreço
2,Inauguração Farmácias Unipreço Sitio Cercado,17 mil visualizações,4,28 de jan. de 2022,Transcrição,Farmácias Unipreço
3,Polivitamínico Politabs Femme - Farmácias Unip...,13 mil visualizações,91,19 de ago. de 2016,NaN,Farmácias Unipreço
4,Inauguração Farmácias Unipreço IPO Água Verde,12 mil visualizações,2,28 de jan. de 2022,Transcrição,Farmácias Unipreço


In [8]:

top_videos.assign(
    date=lambda df_: df_["date"].apply(trata_data),
    views=lambda df_: df_["views"].apply(trata_visualizacoes),
    # comments_amount = lambda df_: df_["comments_amount"].apply(trata_comments),
    category="TOP",
).to_parquet(base_path + "top_videos.parquet")


In [9]:


top_videos_silver = pd.read_parquet(base_path + "top_videos.parquet")



In [12]:

# # Trata lifetime


df_lifetime = pd.read_csv(f"{base_path}lifetime.csv")


df_lifetime.head()



def days_on_youtube(value) -> int:
    import datetime

    today = datetime.date.today()
    return (today - value.date()).days



df_lifetime = (
    df_lifetime.assign(
        date_joined=lambda df_: df_["date_joined"].apply(trata_data),
        total_views=lambda df_: df_["total_views"].apply(trata_visualizacoes),
        total_videos = lambda d: d['total_videos'].apply(trata_visualizacoes),
        total_subs=lambda df_: df_["total_subs"].apply(trata_inscritos),
        days_since_joined=lambda df_: df_["date_joined"].apply(days_on_youtube),
    )
)#.to_csv(base_path + "lifetime_clean.csv", index=False)




In [13]:
df_lifetime

,channel_title,date_joined,total_views,total_subs,total_videos,days_since_joined
0,Farmácias Unipreço,2011-11-16,196650,242,79,4345
1,Farmácias Nissei,2013-01-22,1317082,1160,169,3912
2,Farmácias Pague Menos,2013-11-29,53283388,248000,7500,3601
3,Drogaria São Paulo,2013-10-01,8302275,25600,161,3660
4,Panvel Farmácias,2009-04-08,22673817,9880,106,5297


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [14]:
lifetime_csv = pd.DataFrame()

lifetime_csv['Channel Id'] = np.arange(5)

lifetime_csv[' Channel Name'] = df_lifetime['channel_title']
lifetime_csv[' Days on YouTube'] = df_lifetime['days_since_joined']
lifetime_csv[' Total Views'] = df_lifetime['total_views']
lifetime_csv[' Views per Day'] = lifetime_csv[' Total Views'] / lifetime_csv[' Days on YouTube']
lifetime_csv[' Total Subscribers'] = df_lifetime['total_subs']
lifetime_csv[' Subscribers per Day'] = lifetime_csv[' Total Subscribers'] / lifetime_csv[' Days on YouTube']
lifetime_csv[' Total Uploads'] = df_lifetime['total_videos']
lifetime_csv[' Uploads per Month'] = lifetime_csv[' Total Uploads'] / (lifetime_csv[' Days on YouTube'] / 30)
lifetime_csv[''] = np.arange(5)

lifetime_csv.to_csv(base_path + "Lifetime.csv", index=False, decimal=',')

In [15]:
# # join


videos = pd.read_parquet(base_path + "videos.parquet")
top_videos = pd.read_parquet(base_path + "top_videos.parquet")

videos = (
    videos.assign(
        title_canal = lambda d: d['channel_name'] + ' - ' + d['title']
    )
)

top_videos = (
    top_videos.assign(
        title_canal = lambda d: d['channel_name'] + ' - ' + d['title']
    )
)
    


# videos.sort_values("channel_name").to_csv(base_path + "videos_clean.csv", index=False, sep=";")
# top_videos.sort_values("channel_name").to_csv(base_path + "top_videos_clean.csv", index=False, sep=";")


# # join the two dataframes

# videos = pd.concat([videos, top_videos])


# videos_sorted = videos.sort_values("channel_name")

# videos_sorted["subscribers"] = videos_sorted["subscribers"].fillna(method="ffill")


# videos_sorted = videos_sorted.sort_values(["channel_name", "category", "date"])


# videos_sorted.to_parquet(base_path + "videos_sorted.parquet")


# (pd.read_parquet(base_path + "videos_sorted.parquet")
#  .assign(title_canal = lambda d: d['channel_name'] + ' - ' + d['title'])
#  .to_csv(
#     base_path + "videos_sorted.csv", index=False, sep=';'
# ))

In [16]:
videos_aux = pd.DataFrame()


videos_aux['Channel Id'] = np.arange(1, len(videos) + 1)
videos_aux[' Channel Name'] = videos['channel_name']
videos_aux[' Video Id'] = np.arange(1, len(videos) + 1)
videos_aux[' VideoUrl'] = np.arange(1, len(videos) + 1)
videos_aux[' Title'] = videos['title_canal']
videos_aux[' Published Date'] = videos['date']
videos_aux[' Views'] = videos['views']
videos_aux[' Likes'] = videos['likes']
videos_aux[' Dislikes'] = np.arange(1, len(videos) + 1)
videos_aux[' Comments'] = videos['comments_amount']
videos_aux[' Engagement %'] = np.arange(1, len(videos) + 1)

In [17]:
videos_aux.to_csv(base_path + "RecentUploads.csv", index=False)

In [18]:
top_videos.head()

,title,views,likes,date,comments_amount,channel_name,category,title_canal
0,CHEGAMOS UNIPREÇO SÍTIO CERCADO,51000,0,2022-09-20,None,Farmácias Unipreço,TOP,Farmácias Unipreço - CHEGAMOS UNIPREÇO SÍTIO C...
1,Inauguração Mega Store Paranaguá,22000,5,2021-09-01,None,Farmácias Unipreço,TOP,Farmácias Unipreço - Inauguração Mega Store Pa...
2,Inauguração Farmácias Unipreço Sitio Cercado,17000,4,2022-01-28,Transcrição,Farmácias Unipreço,TOP,Farmácias Unipreço - Inauguração Farmácias Un...
3,Polivitamínico Politabs Femme - Farmácias Unip...,13000,91,2016-08-19,None,Farmácias Unipreço,TOP,Farmácias Unipreço - Polivitamínico Politabs F...
4,Inauguração Farmácias Unipreço IPO Água Verde,12000,2,2022-01-28,Transcrição,Farmácias Unipreço,TOP,Farmácias Unipreço - Inauguração Farmácias Uni...


In [19]:
videos_aux = pd.DataFrame()


videos_aux['Channel Id'] = np.arange(1, len(videos) + 1)
videos_aux[' Channel Name'] = top_videos['channel_name']
videos_aux[' Video Id'] = np.arange(1, len(videos) + 1)
videos_aux[' VideoUrl'] = np.arange(1, len(videos) + 1)
videos_aux[' Title'] = top_videos['title_canal']
videos_aux[' Published Date'] = top_videos['date']
videos_aux[' Views'] = top_videos['views']
videos_aux[' Likes'] = top_videos['likes']
videos_aux[' Dislikes'] = np.arange(1, len(videos) + 1)
videos_aux[' Comments'] = top_videos['comments_amount']
videos_aux[' Engagement %'] = np.arange(1, len(videos) + 1)

In [20]:
videos_aux.to_csv(base_path + "TopUploads.csv", index=False)

last 30 days

In [21]:
# read channel IDs

with open("canais.txt", "r") as f:
    canais = f.readlines()


canais = [c.strip() for c in canais]
# take only the string after the last slash
canais = [c.split("/")[-1] for c in canais]

# print(canais)

base_url = "https://socialblade.com/youtube/channel/"

last_30days = pd.read_parquet(f'data/{canais[0].split("@")[-1]}/last_30_days.parquet')


def fix_views(view: pd.Series) -> pd.Series:
    view = view.str.replace("+", "").str.replace(",", "")
    # print(view)

    view = view.apply(lambda x: float(x.replace("K", "")) * 1000 if "K" in x else x)
    view = view.apply(
        lambda x: float(x.replace("M", "")) * 1000000 if "M" in str(x) else x
    )

    return view


last_30days = last_30days.assign(
    views=lambda x: fix_views(x["views"]), 
    subs=lambda x: fix_views(x["subs"])
)

# Replace '---' with NaN in the 'subs' column
last_30days['subs'] = last_30days['subs'].replace('--', np.nan)



In [22]:
last_30days = last_30days.astype({'views': 'float64', 'subs': 'float64'})


In [23]:
last_30_days_aux = pd.DataFrame()



last_30_days_aux['Channel Id'] = np.arange(5)
last_30_days_aux[' Channel Name'] = df_lifetime['channel_title']
last_30_days_aux['  Views Last 30 Days'] = last_30days['views']
last_30_days_aux[' Views per Day'] = last_30_days_aux['  Views Last 30 Days']/30
last_30_days_aux[' Total Subscribers Last 30 Days'] = last_30days['subs']
last_30_days_aux[' Subscribers per Day'] = last_30_days_aux[' Total Subscribers Last 30 Days']/30
last_30_days_aux[' Total Uploads Last 30 Days'] = ["ATUALIZAR", "ATUALIZAR", "ATUALIZAR", "ATUALIZAR", "ATUALIZAR"]
last_30_days_aux[' Uploads per Week'] = np.arange(5)
last_30_days_aux[''] = np.arange(5)








In [24]:
last_30_days_aux

,Channel Id,Channel Name,Views Last 30 Days,Views per Day,Total Subscribers Last 30 Days,Subscribers per Day,Total Uploads Last 30 Days,Uploads per Week,
0,0,Farmácias Unipreço,0.0,0.000000,0.0,0.000000,ATUALIZAR,0,0
1,1,Farmácias Nissei,0.0,0.000000,0.0,0.000000,ATUALIZAR,1,1
2,2,Farmácias Pague Menos,3649000.0,121633.333333,12000.0,400.000000,ATUALIZAR,2,2
3,3,Drogaria São Paulo,22888.0,762.933333,300.0,10.000000,ATUALIZAR,3,3
4,4,Panvel Farmácias,6222.0,207.400000,100.0,3.333333,ATUALIZAR,4,4


In [25]:
last_30_days_aux.to_csv(base_path + "Last30Days.csv", index=False, decimal=',')

In [26]:

# last_30days.to_csv(f'data/{canais[0].split("@")[-1]}/last_30days.csv', index=False)